<a href="https://colab.research.google.com/github/Aligreu/Deep-Learning-with-Tensorflow-and-Pytorch-Projects/blob/main/Fashion_Mnist_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
from torchvision import transforms as T
from torchvision import datasets
from torch.utils.data import DataLoader

In [9]:
transforms = T.Compose([
    T.ToTensor()
])

train_data = datasets.FashionMNIST(root='./data', download=True, train=True, transform=transforms)
test_data = datasets.FashionMNIST(root='./data', download=True, train=False, transform=transforms)

In [13]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [14]:
train_data.class_to_idx

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

In [8]:
train_loader = DataLoader(dataset=train_data, shuffle=True, batch_size=64)
test_loader = DataLoader(dataset=test_data, shuffle=False, batch_size=64)

In [58]:
class FMinestModel(nn.Module):
    def __init__(self, num_classes):
        super(FMinestModel, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1024, kernel_size=3, padding=1, stride=1),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(512*7*7, 128),
            nn.Linear(128, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        pred = self.model(x)
        return pred

model = FMinestModel(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

FMinestModel(
  (model): Sequential(
    (0): Conv2d(1, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=25088, out_features=128, bias=True)
    (8): Linear(in_features=128, out_features=10, bias=True)
    (9): LogSoftmax(dim=1)
  )
)

In [59]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for image, label in train_loader:
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch+1} Loss: {loss.item()}")

Epoch: 1 Loss: 0.7626997232437134
Epoch: 2 Loss: 0.45961901545524597
Epoch: 3 Loss: 0.23228393495082855
Epoch: 4 Loss: 0.6038681268692017
Epoch: 5 Loss: 0.33818942308425903
Epoch: 6 Loss: 0.45754167437553406
Epoch: 7 Loss: 0.36478331685066223
Epoch: 8 Loss: 0.5434313416481018
Epoch: 9 Loss: 0.2663448452949524
Epoch: 10 Loss: 0.17990180850028992
Epoch: 11 Loss: 0.5507586598396301
Epoch: 12 Loss: 0.28224602341651917
Epoch: 13 Loss: 0.32103896141052246
Epoch: 14 Loss: 0.20860978960990906
Epoch: 15 Loss: 0.5006498098373413
Epoch: 16 Loss: 0.17793002724647522
Epoch: 17 Loss: 0.6814471483230591
Epoch: 18 Loss: 0.27835598587989807
Epoch: 19 Loss: 0.33942002058029175
Epoch: 20 Loss: 0.4861607849597931


In [49]:
!pip install torchmetrics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 11.5 MB/s eta 0:00:00


In [61]:
from torchmetrics  import Accuracy

accuracy = Accuracy(task='multiclass', num_classes=10).to(device)

with torch.no_grad():
    model.eval()
    for image, label in test_loader:
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        outputs = torch.exp(outputs)
        preds = torch.argmax(outputs, dim=1)
        accuracy.update(preds, label)

print(f"Accuracy {accuracy.compute():.4f}")

Accuracy 0.8678
